In [296]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [297]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import numpy as np
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [298]:
train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")

In [299]:
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

In [300]:
pd.set_option('display.max_columns', None)
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [301]:
test.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [302]:
train.describe()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [303]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [304]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   787511 non-null  float64
 2   Gender                800000 non-null  object 
 3   Annual Income         770140 non-null  float64
 4   Marital Status        787664 non-null  object 
 5   Number of Dependents  726870 non-null  float64
 6   Education Level       800000 non-null  object 
 7   Occupation            560875 non-null  object 
 8   Health Score          750551 non-null  float64
 9   Location              800000 non-null  object 
 10  Policy Type           800000 non-null  object 
 11  Previous Claims       557198 non-null  float64
 12  Vehicle Age           799997 non-null  float64
 13  Credit Score          708549 non-null  float64
 14  Insurance Duration    799998 non-null  float64
 15  

In [305]:
train.isnull().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [306]:
test.isnull().sum()

id                           0
Age                      12489
Gender                       0
Annual Income            29860
Marital Status           12336
Number of Dependents     73130
Education Level              0
Occupation              239125
Health Score             49449
Location                     0
Policy Type                  0
Previous Claims         242802
Vehicle Age                  3
Credit Score             91451
Insurance Duration           2
Policy Start Date            0
Customer Feedback        52276
Smoking Status               0
Exercise Frequency           0
Property Type                0
dtype: int64

**Pre-Processing**

In [307]:
train['Policy Start Date'] = pd.to_datetime(train['Policy Start Date'], errors='coerce')

In [308]:
test['Policy Start Date'] = pd.to_datetime(train['Policy Start Date'], errors='coerce')

In [309]:
train['Education Level'] = train['Education Level'].replace({"Bachelor's" : "Bachelors"})
train['Education Level'] = train['Education Level'].replace({"Master's" : "Masters"})

In [310]:
test['Education Level'] = test['Education Level'].replace({"Bachelor's" : "Bachelors"})
test['Education Level'] = test['Education Level'].replace({"Master's" : "Masters"})

In [311]:
def detect_outliers(train, test, target_column):
    numerical_cols_train = train.select_dtypes(include=['int64', 'float64']).columns
    numerical_cols_train = [col for col in numerical_cols_train if col != target_column]

    numerical_cols_test = test.select_dtypes(include=['int64', 'float64']).columns

    outlier_indices_train = []
    outlier_indices_test = []

    for col in numerical_cols_train:
        if col in train.columns:
            Q1 = train[col].quantile(0.25)
            Q3 = train[col].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers_col_train = train[(train[col] < lower_bound) | (train[col] > upper_bound)]
            outlier_indices_train.extend(outliers_col_train.index)
    
    if target_column in train.columns:
        Q1 = train[target_column].quantile(0.25)
        Q3 = train[target_column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_col_train = train[(train[target_column] < lower_bound) | (train[target_column] > upper_bound)]
        outlier_indices_train.extend(outliers_col_train.index)

    outliers_train = train.loc[outlier_indices_train]
    
    for col in numerical_cols_test:
        if col in test.columns:  
            Q1 = test[col].quantile(0.25)
            Q3 = test[col].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers_col_test = test[(test[col] < lower_bound) | (test[col] > upper_bound)]
            outlier_indices_test.extend(outliers_col_test.index)
    
    outliers_test = test.loc[outlier_indices_test]
    
    return outliers_train, outliers_test

target_column = 'Premium Amount'
outliers_train, outliers_test = detect_outliers(train, test, target_column)

In [312]:
outlier_proportion = len(outliers_train) / len(train) * 100
print(f"Outlier Proportion: {outlier_proportion}%")

Outlier Proportion: 9.735083333333334%


In [313]:
outlier_proportion = len(outliers_test) / len(test) * 100
print(f"Outlier Proportion: {outlier_proportion}%")

Outlier Proportion: 5.647625%


In [314]:
def new_datetime_features(df, date_column):
    df[f'{date_column}_year'] = df[date_column].dt.year
    df[f'{date_column}_month'] = df[date_column].dt.month
    df[f'{date_column}_day'] = df[date_column].dt.day
    df[f'{date_column}_weekday'] = df[date_column].dt.weekday 
    df[f'{date_column}_hour'] = df[date_column].dt.hour
    df[f'{date_column}_minute'] = df[date_column].dt.minute
    df[f'{date_column}_second'] = df[date_column].dt.second
    df[f'{date_column}_is_weekend'] = df[date_column].dt.weekday >= 5
    return df

train = new_datetime_features(train, 'Policy Start Date')
test = new_datetime_features(test, 'Policy Start Date')

In [315]:
train[['Policy Start Date_year', 'Policy Start Date_month', 'Policy Start Date_day', 
       'Policy Start Date_weekday', 'Policy Start Date_hour', 
       'Policy Start Date_minute', 'Policy Start Date_second']] = \
    train[['Policy Start Date_year', 'Policy Start Date_month', 'Policy Start Date_day', 
           'Policy Start Date_weekday', 'Policy Start Date_hour', 
           'Policy Start Date_minute', 'Policy Start Date_second']].astype(int)

train['Policy Start Date_is_weekend'] = train['Policy Start Date_is_weekend'].astype(bool)

test[['Policy Start Date_year', 'Policy Start Date_month', 'Policy Start Date_day', 
      'Policy Start Date_weekday', 'Policy Start Date_hour', 
      'Policy Start Date_minute', 'Policy Start Date_second']] = \
    test[['Policy Start Date_year', 'Policy Start Date_month', 'Policy Start Date_day', 
          'Policy Start Date_weekday', 'Policy Start Date_hour', 
          'Policy Start Date_minute', 'Policy Start Date_second']].astype(int)

test['Policy Start Date_is_weekend'] = test['Policy Start Date_is_weekend'].astype(bool)


In [316]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 29 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   id                            1200000 non-null  int64         
 1   Age                           1181295 non-null  float64       
 2   Gender                        1200000 non-null  object        
 3   Annual Income                 1155051 non-null  float64       
 4   Marital Status                1181471 non-null  object        
 5   Number of Dependents          1090328 non-null  float64       
 6   Education Level               1200000 non-null  object        
 7   Occupation                    841925 non-null   object        
 8   Health Score                  1125924 non-null  float64       
 9   Location                      1200000 non-null  object        
 10  Policy Type                   1200000 non-null  object        
 11

In [317]:
train['Age_category'] = pd.cut(train['Age'], bins=[0, 18, 35, 50, 100], labels=["Child", "Young-Adult", "Middle-Aged", "Senior"])
test['Age_category'] = pd.cut(test['Age'], bins=[0, 18, 35, 50, 100], labels=["Child", "Young-Adult", "Middle-Aged", "Senior"])

In [318]:
train["Income_to_Credit_Score_Ratio"] = train["Annual Income"] / train["Credit Score"]
test["Income_to_Credit_Score_Ratio"] = test["Annual Income"] / test["Credit Score"]

In [319]:
train['Vehicle_Age_Group'] = pd.cut(train['Vehicle Age'], bins=[0, 2, 5, 10, 20, 100], labels=['New', 'Relatively New', 'Middle-aged', 'Old', 'Very Old'])
test['Vehicle_Age_Group'] = pd.cut(test['Vehicle Age'], bins=[0, 2, 5, 10, 20, 100], labels=['New', 'Relatively New', 'Middle-aged', 'Old', 'Very Old'])

In [320]:
train['Days_Since_Policy_Start'] = (pd.to_datetime('today') - train['Policy Start Date']).dt.days
test['Days_Since_Policy_Start'] = (pd.to_datetime('today') - test['Policy Start Date']).dt.days

In [321]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Policy Start Date_year,Policy Start Date_month,Policy Start Date_day,Policy Start Date_weekday,Policy Start Date_hour,Policy Start Date_minute,Policy Start Date_second,Policy Start Date_is_weekend,Age_category,Income_to_Credit_Score_Ratio,Vehicle_Age_Group,Days_Since_Policy_Start
0,0,19.0,Female,10049.0,Married,1.0,Bachelors,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0,2023,12,23,5,15,21,39,True,Young-Adult,27.013441,Old,371
1,1,39.0,Female,31678.0,Divorced,3.0,Masters,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0,2023,6,12,0,15,21,39,False,Middle-Aged,45.645533,Old,565
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0,2023,9,30,5,15,21,39,True,Young-Adult,NaN,Old,455
3,3,21.0,Male,141855.0,Married,2.0,Bachelors,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0,2024,6,12,2,15,21,39,False,Young-Adult,386.525886,NaN,199
4,4,21.0,Male,39651.0,Single,1.0,Bachelors,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0,2021,12,1,2,15,21,39,False,Young-Adult,66.306020,Middle-aged,1123


### MODEL

In [322]:
X = train.drop(columns=["Premium Amount", "id"])
y = train['Premium Amount']

In [323]:
X_train, X_val, y_train, y_val =  train_test_split(X, y, test_size=0.2, random_state=42)

In [324]:
numerical_features = X.select_dtypes(include=["int64","float64"]).columns
categorical_features = X.select_dtypes(include=["object", "category"]).columns

In [325]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [326]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [327]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [328]:
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

In [329]:
xgb_model = XGBRegressor(
    n_estimators=100,  
    max_depth=6,       
    learning_rate=0.1, 
    subsample=0.8,     
    colsample_bytree=0.8,  
    random_state=42    
)

In [330]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('regressor', xgb_model)         
])

In [331]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Policy Start Date_year', 'Policy Start Date_month',
       'Policy Start Date_day', 'Policy St...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [332]:
y_pred = pipeline.predict(X_val)

In [333]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

rmsle = np.sqrt(mean_squared_log_error(y_val, y_pred))

print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle}")

Root Mean Squared Logarithmic Error (RMSLE): 1.1422750587726693


In [334]:
X_test = test.drop(columns=['id'])

In [335]:
X_test_processed = preprocessor.transform(X_test)

In [337]:
y_test_pred = xgb_model.predict(X_test_processed)

In [338]:
submission = pd.DataFrame({
    'id': test['id'],  
    'Premium Amount': y_test_pred
})

In [339]:
submission.head()

,id,Premium Amount
0,1200000,1407.799438
1,1200001,1066.477417
2,1200002,1081.199707
3,1200003,1092.438965
4,1200004,1023.256104


In [340]:
submission.to_csv('submission.csv', index=False)